# Обогащение данных

1. Заполнить отсутствующие значения по погоде интерполяционными данными.
2. Для точки росы вычесть температуру воздуха.
3. Напрвыление ветра разложить на синус и косинус.
4. Для тем. воздуха вычеслить 1-ю и 2-ю производные.
5. Вывести параметры по праздничным дням, дням недели, месяцам и неделям года.
6. Посчитать модель линейной регрессии по первым 20 зданиям и найти точность.

In [1]:
import pandas as pd
import numpy as np

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

## Загрузка данных

In [2]:
buildings = pd.read_csv("../data/buildings.csv")
weather = pd.read_csv("../data/weather.csv")
energy = pd.read_csv("../data/train.0.csv")

## Отобрать 20 зданий и объединить

In [7]:
energy = energy[energy["building_id"] < 20]
energy = pd.merge(
    left=energy,
    right=buildings,
    how="left",
    left_on="building_id",
    right_on="building_id",
)
del buildings
print(energy.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175680 entries, 0 to 175679
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   building_id    175680 non-null  int64  
 1   meter          175680 non-null  int64  
 2   timestamp      175680 non-null  object 
 3   meter_reading  175680 non-null  float64
 4   site_id        175680 non-null  int64  
 5   primary_use    175680 non-null  object 
 6   square_feet    175680 non-null  int64  
 7   year_built     175680 non-null  float64
 8   floor_count    0 non-null       float64
dtypes: float64(3), int64(4), object(2)
memory usage: 13.4+ MB
None


## Интерполяция значений для weather

In [9]:
weather = weather[weather["site_id"] == 0]
weather["precip_depth_1_hr"] = weather["precip_depth_1_hr"].apply(lambda x: x if x > 0 else 0)
interpolate_columns = [
    "air_temperature",
    "dew_temperature",
    "cloud_coverage",
    "wind_speed",
    "wind_direction",
    "precip_depth_1_hr",
    "sea_level_pressure",
]

for col in interpolate_columns:
    weather[col] = weather[col].interpolate(limit_direction="both", kind="cubic")